In [ ]:
import glob
import json
import pandas as pd
import re

def get_counts(emoji_list):
    d = {}
    for emoji in emoji_list:
        if emoji in d:
            d[emoji] += 1
        else:
            d[emoji] = 1
    return d

def get_emoji_files(channel):
    dir_path = f'awakened_zip/{channel}'
    files = glob.glob(f"{dir_path}/*.json")
    return files

def get_emojis(emoji_file):
    j = json.load(open(emoji_file))

    messages = []
    for message in j:
        text = message['text']
        if 'reactions' in message:
            reactions = []
            for reaction in message['reactions']:
                reactions.append(f":{reaction['name']}:")
        else:
            reactions = []

        msg = {
            'text': text,
            'reactions': reactions,
        }
        messages.append(msg)

    df = pd.DataFrame(messages)

    def extract_emojis(text):
        return re.findall(
            r':[a-z0-9_-]+:',
            text
        )

    emojis = [emoji for emoji_list in df.text.apply(extract_emojis) for emoji in emoji_list]
    for reactions in df.reactions:
        emojis.extend(reactions)

    return emojis

def get_emoji_counts(channel, head=-1):
    emoji_files = get_emoji_files(channel)
    emoji_list = []
    for emoji_file in emoji_files[:head]:
        emojis = get_emojis(emoji_file)
        emoji_list.extend(emojis)
    
    emoji_counts = get_counts(emoji_list)
    return emoji_counts

# Time

1829 total

- 50 -> total: 436 ms
- 1829 -> 15s

In [ ]:
counts = get_emoji_counts('chopping-wood')

In [ ]:
import pandas as pd

s = pd.Series(counts).sort_values(ascending=False)
df = s.to_frame().reset_index()
df.columns = ['emoji_name', 'count']

df.head()

In [ ]:
for i, row in df.sort_values('count', ascending=False).iterrows():
    print(row)

In [ ]:
df.to_csv('data/emoji-names-and-count.csv', index=False)

In [ ]:
!head data/emoji-names-and-count.csv